###### Imports and Settings

In [31]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [32]:
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces

# American Community Survey 2017-2021 5 Year Estimates

In [33]:
#read in API key
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [34]:
#variable containing Census API key
api_key = keys_dict_2['CENSUS']

## Read In Data Guide

In [35]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE ACS 2016 CEDS.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [36]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]
dg3 = dataguide[dataguide['ID'].between(93, 138)]
dg4 = dataguide[dataguide['ID'].between(139, 184)]
dg5 = dataguide[dataguide['ID'].between(185, 230)]
dg6 = dataguide[dataguide['ID'].between(231, 276)]
dg7 = dataguide[dataguide['ID'].between(277, 322)]
dg8 = dataguide[dataguide['ID'].between(323, 368)]
#dg9 = dataguide[dataguide['ID'].between(369, 414)]

In [37]:
dfs = [dg1, dg2, dg3, dg4, dg5, dg6, dg7, dg8, dg9]

In [38]:
#url string and list parameters for column head and tail
url_str= 'https://api.census.gov/data/2016/acs/acs5?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
tail_cols3 = 'GeoFIPS2'

In [39]:
results = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list.append(tail_cols2)
        col_list = deque(col_list)
        col_list.appendleft(head2)
        col_list.appendleft(head1)
        predicates= {} #call for all counties in state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "county:*"
        predicates["in"]= "state:47"                                                             
        data = requests.get(url_str, params= predicates)                                                                
        col_names = col_list
        df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
        col_list.remove(tail_cols2) #adjust the column list for different formats of geos
        predicates= {} #call for state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "state:47"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, state], axis = 0)
        predicates= {} #call for US
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "us:*"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, national], axis = 0)
        results.append(df)
new_df = pd.concat(results, axis = 1)
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS', 'GeoFIPS']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)
print('Okay Finished')

Okay Finished


In [40]:
data = new_df

In [41]:
data.head()

,NAME,GEO_ID,attainment_total_over25_series,attainment_noschooling,attainment_nurseryschool,attainment_kindergarten,attainment_1stgrade,attainment_2ndgrade,attainment_3rdgrade,attainment_4thgrade,attainment_5thgrade,attainment_6thgrade,attainment_7thgrade,attainment_8thgrade,attainment_9thgrade,attainment_10thgrade,attainment_11thgrade,attainment_12thgradenodiploma,attainment_regularhighschooldiploma,attainment_gedoralternativecredential,attainment_somecollegelessthan1year,attainment_somecollege1ormoreyearsnodegree,attainment_associatesdegree,attainment_bachelorsdegree,attainment_mastersdegree,attainment_professionalschooldegree,attainment_doctoratedegree,lfstatus_total_sexbyagebyemploymentstatus16+_series,lfstatus_m_total,lfstatus_m_16to19,lfstatus_m_16to19_inlf,lfstatus_m_16to19_inlf_armedforces,lfstatus_m_16to19_inlf_civilian,lfstatus_m_16to19_inlf_civilian_employed,lfstatus_m_16to19_inlf_civilian_unemployed,lfstatus_m_16to19_notinlf,lfstatus_m_20to21,lfstatus_m_20to21_inlf,lfstatus_m_20to21_inlf_armedforces,lfstatus_m_20to21_inlf_civilian,lfstatus_m_20to21_inlf_civilian_employed,lfstatus_m_20to21_inlf_civilian_unemployed,lfstatus_m_20to21_notinlf,lfstatus_m_22to24,lfstatus_m_22to24_inlf,lfstatus_m_22to24_inlf_armedforces,lfstatus_m_22to24_inlf_civilian,lfstatus_m_22to24_inlf_civilian_employed,lfstatus_m_22to24_inlf_civilian_unemployed,lfstatus_m_22to24_notinlf,lfstatus_m_25to29,lfstatus_m_25to29_inlf,lfstatus_m_25to29_inlf_armedforces,lfstatus_m_25to29_inlf_civilian,lfstatus_m_25to29_inlf_civilian_employed,lfstatus_m_25to29_inlf_civilian_unemployed,lfstatus_m_25to29_notinlf,lfstatus_m_30to34,lfstatus_m_30to34_inlf,lfstatus_m_30to34_inlf_armedforces,lfstatus_m_30to34_inlf_civilian,lfstatus_m_30to34_inlf_civilian_employed,lfstatus_m_30to34_inlf_civilian_unemployed,lfstatus_m_30to34_notinlf,lfstatus_m_35to44,lfstatus_m_35to44_inlf,lfstatus_m_35to44_inlf_armedforces,lfstatus_m_35to44_inlf_civilian,lfstatus_m_35to44_inlf_civilian_employed,lfstatus_m_35to44_inlf_civilian_unemployed,lfstatus_m_35to44_notinlf,lfstatus_m_45to54,lfstatus_m_45to54_inlf,lfstatus_m_45to54_inlf_armedforces,lfstatus_m_45to54_inlf_civilian,lfstatus_m_45to54_inlf_civilian_employed,lfstatus_m_45to54_inlf_civilian_unemployed,lfstatus_m_45to54_notinlf,lfstatus_m_55to59,lfstatus_m_55to59_inlf,lfstatus_m_55to59_inlf_armedforces,lfstatus_m_55to59_inlf_civilian,lfstatus_m_55to59_inlf_civilian_employed,lfstatus_m_55to59_inlf_civilian_unemployed,lfstatus_m_55to59_notinlf,lfstatus_m_60to61,lfstatus_m_60to61_inlf,lfstatus_m_60to61_inlf_armedforces,lfstatus_m_60to61_inlf_civilian,lfstatus_m_60to61_inlf_civilian_employed,lfstatus_m_60to61_inlf_civilian_unemployed,lfstatus_m_60to61_notinlf,lfstatus_m_62to64,lfstatus_m_62to64_inlf,lfstatus_m_62to64_inlf_armedforces,lfstatus_m_62to64_inlf_civilian,lfstatus_m_62to64_inlf_civilian_employed,lfstatus_m_62to64_inlf_civilian_unemployed,lfstatus_m_62to64_notinlf,lfstatus_m_65to69,lfstatus_m_65to69_inlf,lfstatus_m_65to69_inlf_employed,lfstatus_m_65to69_inlf_unemployed,lfstatus_m_65to69_notinlf,lfstatus_m_70to74,lfstatus_m_70to74_inlf,lfstatus_m_70to74_inlf_employed,lfstatus_m_70to74_inlf_unemployed,lfstatus_m_70to74_notinlf,lfstatus_m_75andolder,lfstatus_m_75andolder_inlf,lfstatus_m_75andolder_inlf_employed,lfstatus_m_75andolder_inlf_unemployed,lfstatus_m_75andolder_notinlf,lfstatus_f_total,lfstatus_f_16to19,lfstatus_f_16to19_inlf,lfstatus_f_16to19_inlf_armedforces,lfstatus_f_16to19_inlf_civilian,lfstatus_f_16to19_inlf_civilian_employed,lfstatus_f_16to19_inlf_civilian_unemployed,lfstatus_f_16to19_notinlf,lfstatus_f_20to21,lfstatus_f_20to21_inlf,lfstatus_f_20to21_inlf_armedforces,lfstatus_f_20to21_inlf_civilian,lfstatus_f_20to21_inlf_civilian_employed,lfstatus_f_20to21_inlf_civilian_unemployed,lfstatus_f_20to21_notinlf,lfstatus_f_22to24,lfstatus_f_22to24_inlf,lfstatus_f_22to24_inlf_armedforces,lfstatus_f_22to24_inlf_civilian,lfstatus_f_22to24_inlf_civilian_employed,lfstatus_f_22to24_inlf_civilian_unemployed,lfstatus_f_22to24_notin

In [42]:
data = data.set_index(['NAME', 'GEO_ID']).transpose()
data.head()

NAME,"Sumner County, Tennessee","Trousdale County, Tennessee","Davidson County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Cheatham County, Tennessee","Williamson County, Tennessee","Humphreys County, Tennessee","Stewart County, Tennessee","Dickson County, Tennessee","Rutherford County, Tennessee","Wilson County, Tennessee","Houston County, Tennessee","Maury County, Tennessee",Tennessee,United States
GEO_ID,0500000US47165,0500000US47169,0500000US47037,0500000US47125,0500000US47147,0500000US47021,0500000US47187,0500000US47085,0500000US47161,0500000US47043,0500000US47149,0500000US47189,0500000US47083,0500000US47119,0400000US47,0100000US
attainment_total_over25_series,116882,5339,454177,115734,45660,27138,133044,12762,9370,34804,179453,85573,5727,58817,4427340,213649147
attainment_noschooling,1046,133,7168,1027,391,151,499,159,110,428,1748,725,84,585,50460,3042019
attainment_nurseryschool,28,0,36,0,12,0,0,0,0,0,0,0,0,0,464,35164
attainment_kindergarten,39,0,86,28,0,0,28,0,0,11,13,14,0,0,716,44216
attainment_1stgrade,57,4,123,42,0,0,0,0,0,0,18,38,0,17,1615,136695


In [43]:
numcols = list(data.columns)
numcols
data[numcols] = data[numcols].astype(float)

In [44]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)

In [45]:
data = data.transpose().reset_index()
data.head()

,NAME,GEO_ID,attainment_total_over25_series,attainment_noschooling,attainment_nurseryschool,attainment_kindergarten,attainment_1stgrade,attainment_2ndgrade,attainment_3rdgrade,attainment_4thgrade,attainment_5thgrade,attainment_6thgrade,attainment_7thgrade,attainment_8thgrade,attainment_9thgrade,attainment_10thgrade,attainment_11thgrade,attainment_12thgradenodiploma,attainment_regularhighschooldiploma,attainment_gedoralternativecredential,attainment_somecollegelessthan1year,attainment_somecollege1ormoreyearsnodegree,attainment_associatesdegree,attainment_bachelorsdegree,attainment_mastersdegree,attainment_professionalschooldegree,attainment_doctoratedegree,lfstatus_total_sexbyagebyemploymentstatus16+_series,lfstatus_m_total,lfstatus_m_16to19,lfstatus_m_16to19_inlf,lfstatus_m_16to19_inlf_armedforces,lfstatus_m_16to19_inlf_civilian,lfstatus_m_16to19_inlf_civilian_employed,lfstatus_m_16to19_inlf_civilian_unemployed,lfstatus_m_16to19_notinlf,lfstatus_m_20to21,lfstatus_m_20to21_inlf,lfstatus_m_20to21_inlf_armedforces,lfstatus_m_20to21_inlf_civilian,lfstatus_m_20to21_inlf_civilian_employed,lfstatus_m_20to21_inlf_civilian_unemployed,lfstatus_m_20to21_notinlf,lfstatus_m_22to24,lfstatus_m_22to24_inlf,lfstatus_m_22to24_inlf_armedforces,lfstatus_m_22to24_inlf_civilian,lfstatus_m_22to24_inlf_civilian_employed,lfstatus_m_22to24_inlf_civilian_unemployed,lfstatus_m_22to24_notinlf,lfstatus_m_25to29,lfstatus_m_25to29_inlf,lfstatus_m_25to29_inlf_armedforces,lfstatus_m_25to29_inlf_civilian,lfstatus_m_25to29_inlf_civilian_employed,lfstatus_m_25to29_inlf_civilian_unemployed,lfstatus_m_25to29_notinlf,lfstatus_m_30to34,lfstatus_m_30to34_inlf,lfstatus_m_30to34_inlf_armedforces,lfstatus_m_30to34_inlf_civilian,lfstatus_m_30to34_inlf_civilian_employed,lfstatus_m_30to34_inlf_civilian_unemployed,lfstatus_m_30to34_notinlf,lfstatus_m_35to44,lfstatus_m_35to44_inlf,lfstatus_m_35to44_inlf_armedforces,lfstatus_m_35to44_inlf_civilian,lfstatus_m_35to44_inlf_civilian_employed,lfstatus_m_35to44_inlf_civilian_unemployed,lfstatus_m_35to44_notinlf,lfstatus_m_45to54,lfstatus_m_45to54_inlf,lfstatus_m_45to54_inlf_armedforces,lfstatus_m_45to54_inlf_civilian,lfstatus_m_45to54_inlf_civilian_employed,lfstatus_m_45to54_inlf_civilian_unemployed,lfstatus_m_45to54_notinlf,lfstatus_m_55to59,lfstatus_m_55to59_inlf,lfstatus_m_55to59_inlf_armedforces,lfstatus_m_55to59_inlf_civilian,lfstatus_m_55to59_inlf_civilian_employed,lfstatus_m_55to59_inlf_civilian_unemployed,lfstatus_m_55to59_notinlf,lfstatus_m_60to61,lfstatus_m_60to61_inlf,lfstatus_m_60to61_inlf_armedforces,lfstatus_m_60to61_inlf_civilian,lfstatus_m_60to61_inlf_civilian_employed,lfstatus_m_60to61_inlf_civilian_unemployed,lfstatus_m_60to61_notinlf,lfstatus_m_62to64,lfstatus_m_62to64_inlf,lfstatus_m_62to64_inlf_armedforces,lfstatus_m_62to64_inlf_civilian,lfstatus_m_62to64_inlf_civilian_employed,lfstatus_m_62to64_inlf_civilian_unemployed,lfstatus_m_62to64_notinlf,lfstatus_m_65to69,lfstatus_m_65to69_inlf,lfstatus_m_65to69_inlf_employed,lfstatus_m_65to69_inlf_unemployed,lfstatus_m_65to69_notinlf,lfstatus_m_70to74,lfstatus_m_70to74_inlf,lfstatus_m_70to74_inlf_employed,lfstatus_m_70to74_inlf_unemployed,lfstatus_m_70to74_notinlf,lfstatus_m_75andolder,lfstatus_m_75andolder_inlf,lfstatus_m_75andolder_inlf_employed,lfstatus_m_75andolder_inlf_unemployed,lfstatus_m_75andolder_notinlf,lfstatus_f_total,lfstatus_f_16to19,lfstatus_f_16to19_inlf,lfstatus_f_16to19_inlf_armedforces,lfstatus_f_16to19_inlf_civilian,lfstatus_f_16to19_inlf_civilian_employed,lfstatus_f_16to19_inlf_civilian_unemployed,lfstatus_f_16to19_notinlf,lfstatus_f_20to21,lfstatus_f_20to21_inlf,lfstatus_f_20to21_inlf_armedforces,lfstatus_f_20to21_inlf_civilian,lfstatus_f_20to21_inlf_civilian_employed,lfstatus_f_20to21_inlf_civilian_unemployed,lfstatus_f_20to21_notinlf,lfstatus_f_22to24,lfstatus_f_22to24_inlf,lfstatus_f_22to24_inlf_armedforces,lfstatus_f_22to24_inlf_civilian,lfstatus_f_22to24_inlf_civilian_employed,lfstatus_f_22to24_inlf_civilian_unemployed,lfstatus_f_22to24_notin

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Columns: 365 entries, NAME to poverty_total_bysexbyage_series
dtypes: float64(363), object(2)
memory usage: 54.3+ KB


In [47]:
data.to_feather('../../Raw Data/ACS2016CEDS')

In [48]:
data.to_csv('../../Raw Data/ACS2016CEDS.csv')